In [1]:
import pandas as pd
import itertools
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
import nltk
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer  
from nltk.corpus import words
from copy import deepcopy
from gensim.models import Word2Vec
from sklearn import svm
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
# load the pretrained tensorflow word2vec embedding
embed = hub.load("https://tfhub.dev/google/Wiki-words-500-with-normalization/2")
embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [21]:
# load the dataset
data = pd.read_csv("movie_data.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [59]:
def text_process(text, stopwords=False, lemmatize=False):
    text = re.sub(r"\d+", "", text.lower())
    
    if stopwords:
        stop_words = set(stopwords.words("english"))
        tokens = word_tokenize(text)
        text_list = [i for i in tokens if not i in stop_words]
        text = " ".join(text_list)
    
    if lemmatize:
        tokens = word_tokenize(text)
        lemmatizer = WordNetLemmatizer()
        text_list = [lemmatizer(word) for word in tokens]
        text = " ".join(text_list)
    
    return text

def split_train(original_train_data, size=5000):
    return original_train_data[:size], original_train_data[size:size+1000]

def binary_transform(sentiment):
    if sentiment == "positive":
        return 1
    return 0

In [67]:
processed = data["review"].apply(text_process,[True, True])
reviews_repr = embed(processed)
label = data["sentiment"].apply(binary_transform)

In [68]:
X_train, X_test = split_train(reviews_repr)
y_train, y_test = split_train(label)

In [126]:
clf = svm.SVC(C=33, kernel="rbf")
clf.fit(X_train, y_train)

SVC(C=33)

In [127]:
prediction = clf.predict(X_test)

In [128]:
count = 0
y = y_test.values
for i in range(len(prediction)):
    if prediction[i] != y[i]:
        count += 1

print(count)
print("Error rate: " + str((count/len(prediction))*100) + "%.")

152
Error rate: 15.2%.
